# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [59]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707323170996_0004,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707323170996_0004,spark,idle,Link,Link,None,✔


In [8]:
val bucketname = "unibo-bd2324-fmagnani"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-fmagnani
path_book_sample: String = s3a://unibo-bd2324-fmagnani/project/book_sample134.json
path_positive_words: String = s3a://unibo-bd2324-fmagnani/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-fmagnani/project/negative-words.txt
res10: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707400852940_0001/


In [9]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [10]:
val bookSampleRdd = spark.read.json(path_book_sample).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [65]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-fmagnani/project/positive-words.txt MapPartitionsRDD[225] at textFile at <console>:24
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-fmagnani/project/negative-words.txt MapPartitionsRDD[227] at textFile at <console>:24


In [5]:
def countOccurrences(src: String, tgt: String): Int =
  src.sliding(tgt.length).count(window => window == tgt)

def countPositive(text: String) = positiveWords.map(word => countOccurrences(text, word)).sum
def countNegative(text: String) = negativeWords.map(word => countOccurrences(text, word)).sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:24: error: not found: value positiveWords
       def countPositive(text: String) = positiveWords.map(word => countOccurrences(text, word)).sum
                                         ^



In [114]:
val positiveDf = positiveWords.toDF("positiveWord")

val reviewsRdd = bookSampleRdd.
    map(review => (review.id, review.reviewText)).
    filter(review => review._2 != null && review._2.length > 5).
    limit(30).
    flatMap({case (id, text) => text.split(" ").map(el => id + "_" + el)}).
    filter(text => text.length > 3).
    toDF("word")

reviewsRdd.join(positiveDf, expr("word like concat('%', positiveWord, '%')")).
    map(row => (row.getAs[String](0), row.getAs[String](1))).
    map({case (el, positiveWord) => (el.substring(0, el.indexOf('_')), 1)}).rdd.
    reduceByKey(_ + _).
    collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveDf: org.apache.spark.sql.DataFrame = [positiveWord: string]
reviewsRdd: org.apache.spark.sql.DataFrame = [word: string]
res202: Array[(String, Int)] = Array((A2PO2J1E3U2WP8-0002005263,7), (A1V1B8FOFE2ZZ-0002005263,24), (A2IUN4UQ9TTQAU-0002005263,8), (A3UHRT62SMS7M7-0002005263,6), (A3Y057QQBRYVI-0001384198,6), (A0334855HN6E38CXWXZR-0002005263,3), (A1WRXACOKPITBQ-0002005263,14), (A25CD3NNU10FXA-0002005263,3), (A3NMH1KTLG7CWX-000047715X,5), (A1BNWEJ7RVPLQ1-0001712799,8), (A27RAHQMS3SSUS-0002005263,1), (ATHTCOG6BB6WK-0001713353,2), (A2Z7OH44GTJ23D-0001384198,6), (A11UA7RCLN6WNU-0002005263,3), (A35NBIN3NJK75S-0001061240,17), (AYIPHN7Q2TBM3-0001061240,3), (AC4I57VYBGO8P-0001713353,5), (A3ABJR7AHYSJYM-0002005263,20), (A1G615KUVK29L6-0001384198,2), (AJAPKRSNE6YIP-0002005263,12), (A2R8RX05RDWD3K-0002005263,8), (A13AL84BGHEF85-0001712799,5), (A1H3A4VJG11WCT-0001061240,5), (A1U11NS6LZ76PB-0002005263,4), (A2YGO6ECA5...
